In [13]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 300)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

startDate = '20201214'
endDate = '20201217'

readPath = r'\\192.168.10.28\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

rawOrderLog = []
for thisPath in dataPathLs:
    data = pd.read_csv(thisPath)
    data = data.rename(columns={'mdClockAtArrival': 'caamd'})
    rawOrderLog += [data]
rawOrderLog = pd.concat(rawOrderLog, sort=False)

# readPath = r'L:\orderLog\data\***'
# dataPathLs = np.array(glob.glob(readPath))
# dateLs = np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])
# dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
# print(dataPathLs)

# rawOrderLog = []
# for thisPath in dataPathLs:
#     data = pickle.load(open(thisPath, 'rb'))
#     data = data.rename(columns={'mdClockAtArrival': 'caamd'})
#     rawOrderLog += [data]
# rawOrderLog = pd.concat(rawOrderLog, sort=False)


for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0).astype('int64')   
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

if rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)].shape[0] != 0:
    display('There are accounts with duplicated ticks:')
    display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
    rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
orderLog = pd.concat([r1, r2])
del r1
del r2    

orderLog = orderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
orderLog['clock'] = orderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
orderLog['broker'] = orderLog['accCode'] // 100
orderLog["broker"] = np.where(orderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), orderLog['accCode'] // 10000, orderLog["broker"])
orderLog['colo_broker'] = orderLog['colo'].str[:2] + '_' + orderLog['broker'].astype('str')
orderLog['order'] = orderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'internalId']).grouper.group_info[0]
orderLog['group'] = orderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
orderLog['orderPrice'] = orderLog['orderPrice'].apply(lambda x: round(x, 2))
orderLog['tradePrice'] = orderLog['tradePrice'].apply(lambda x: round(x, 2))
orderLog['orderDirection1'] = np.where(orderLog["orderDirection"] == -2, -1, np.where(
    orderLog["orderDirection"] == 2, 1, orderLog["orderDirection"]))
orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
df = orderLog.copy()


# ### Assertion 1:  make sure same direction in same date, secid, vai
# print('=======================================================================================')
# print('1. same date, secid, vai: same direction')
# orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
# if len(orderLog[orderLog['directNum'] != 1]) > 0:
#     print('opposite direction for same date, same secid, same vai')
#     display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)].groupby(['accCode'])['orderDirection'].size())
#     orderLog = orderLog[orderLog['directNum'] == 1]

# assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

# ## Assertion 2:  make sure each account, secid, vai only has one insertion
# print('=======================================================================================')
# print('2. same date, secid, vai, accCode: one insertion')
# a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
# if len(a[a > 1]) > 0:
#     print('more than one insertion at same time')
#     a = a[a>1].reset_index()
#     display(a)
#     display(a.groupby(['accCode'])['date'].size())
#     orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

# orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
#                              np.where(orderLog['mse'] == 100, 1, 0), np.nan)
# orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

# placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
# print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


# ### Assertion 3:  check IPO stocks selling status
# print('=======================================================================================')
# print('3. IPO stocks selling (ars = 301, 302)')
# if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
#     kk = orderLog[orderLog['ars'].isin([301, 302])]
#     print(kk)
#     try:
#         assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
#         print('we only sell, never buy')
#     except:
#         print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#         print(kk[kk['orderDirection1'] == 1])
#     kk1 = kk[kk['updateType'] == 0]
#     kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
#     kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
#     kk1['diff'] = kk1['diff'].fillna(0)
#     try:
#         assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
#         print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
#     except:
#         print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#         print(kk1[kk1['diff'] < 10e6])
#     kk2 = kk[(kk['updateType'] == 1)]
#     try:
#         assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
#         print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
#     except:
#         print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#         print(kk2[kk2['duration'] < 3e6])


# ### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
# print('=======================================================================================')
# print('4. updateType 7 orders')
# if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
#     assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().max() < 20)
#     assert(orderLog[orderLog['updateType'] == 7].groupby(['date', 'accCode'])['order'].nunique().sum() < 100)

# ### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
# print('=======================================================================================')
# print('5. updateType 6 orders')
# k1 = orderLog[orderLog['updateType'] == 6].groupby(['date', 'accCode'])['order'].nunique().reset_index()
# k2 = orderLog.groupby(['date', 'accCode'])['order'].nunique().reset_index()
# k = pd.merge(k1, k2, on=['date', 'accCode'], how='left')
# k['prob'] = k['order_x']/k['order_y']
# try:
#     assert(sum(k['prob'] >= 0.05) == 0)
# except:
#     print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#     print(k[k['prob'] >= 0.05])

# ### Assertion 6: check CYB orders, make sure all CYB stocks have absOrderSize < 30w
# print('=======================================================================================')
# print('6. CYB stocks order size < 30w')
# try:
#     cyb = orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)].groupby('date')['absOrderSize'].max().reset_index()
#     assert(cyb[cyb['absOrderSize'] > 300000].shape[0] == 0)
# except:
#     print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    
     
# ### Assertion 7:  make sure there is no unexpected updateType 
# print('=======================================================================================')
# print('7. unexpected updateType')
# def getTuple(x):
#     return tuple(i for i in x)

# checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
# checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
# checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
#                      np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
#                                                                (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
#                                                                (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
#                      np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

# orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
# orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

# ### Assertion 8:  make sure status==0 got all traded
# print('=======================================================================================')
# print('8. status == 0: all traded')
# a = orderLog[orderLog['status'] == 0]
# a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
# a.columns = ['order', 'filled', 'total']
# print('in total trade, any fill != total cases')
# display(a[a['filled'] != a['total']])
# if a[a['filled'] != a['total']].shape[0] > 0:
#     removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
#     orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
# ### Assertion 9:  make sure status==1 got partial traded
# print('=======================================================================================')
# print('9. status == 1: partial traded')
# a = orderLog[orderLog['status'] == 1]
# a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
# a.columns = ['order', 'filled', 'total']
# print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
# display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
# if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
#     removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
#     orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
# ### Assertion 10: make sure no cancellation within 1 sec
# print('=======================================================================================')
# print('10. no cancellation within 1 sec')
# a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
# print('any cancellation within 1 sec')
# display(a)
# if a.shape[0] > 0:
#     removeOrderLs = a['order'].unique()
#     orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


# ### Assertion 11: make sure no order has shares > 80w or notional > 800w
# print('=======================================================================================')
# print('11. Orders with size > 80w or notional > 800w')
# orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
# if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
#     print('some order quantity are > 80w')
#     print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
#     display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
#                                                          'orderNtl', 'orderDirection', 'clock', 'order']])
            
# if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
#     print('some order ntl are > 800w')
#     print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
#     display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
#                                                       'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
#                                                       "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

# removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
# orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


# orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

# orderLog["mrsb90"] = orderLog.groupby(['order'])['mrsb90'].transform('first')
# orderLog["mrss90"] = orderLog.groupby(['order'])['mrss90'].transform('first')
# orderLog["aaa"] = orderLog.groupby(['order'])['aaa'].transform('first')

# orderLog['m1'] = orderLog['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
# orderLog['m2'] = orderLog['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
# if orderLog[orderLog['mrsb90'] == '-'].shape[0] != 0:
#     display(orderLog[orderLog['mrsb90'] == '-'])
# orderLog = orderLog[orderLog['mrsb90'] != '-']
# orderLog['mrsb90'] = orderLog['mrsb90'].astype(float)
# if orderLog[orderLog['aaa'] == '-'].shape[0] != 0:
#     display(orderLog[orderLog['aaa'] == '-'])
# orderLog = orderLog[orderLog['aaa'] != '-']
# orderLog['aaa'] = orderLog['aaa'].astype(float)
# orderLog.loc[(orderLog['orderDirection'] >= 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstauc'] = \
# orderLog.loc[(orderLog['orderDirection'] >= 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm2']

# orderLog.loc[(orderLog['orderDirection'] >= 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'mrstaat'] = \
# orderLog.loc[(orderLog['orderDirection'] >= 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrsb90']) < 1e-12), 'm1']

# orderLog.loc[(orderLog['orderDirection'] < 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstauc'] = \
# orderLog.loc[(orderLog['orderDirection'] < 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm2']

# orderLog.loc[(orderLog['orderDirection'] < 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'mrstaat'] = \
# orderLog.loc[(orderLog['orderDirection'] < 1) &\
#          (orderLog['mrstaat'].isin([11000, 13000])) & (abs(orderLog['aaa'] - orderLog['mrss90']) < 1e-12), 'm1']    

# orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
# orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
# orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
# orderLog["mrstaat"] = orderLog.groupby(['order'])['mrstaat'].transform('first')
# orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
# orderLog["mrstauc"] = orderLog.groupby(['order'])['mrstauc'].transform('first')
# orderLog = orderLog[~orderLog['ars'].isnull()]
# orderLog['sta'] = np.where(orderLog['mrstaat'] == 1000, '1. staone', np.where(
# orderLog['mrstaat'] == 3000, '2. statwo', np.where(
# orderLog['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))
# display(orderLog[orderLog['sta'] == 'else'][['date', 'secid', 'colo', 'accCode', 'updateType', 'vai', 'absOrderSize', 'orderDirection', 'mrstaat', 'mrstauc', 'sdd', 'ApplSeqNum', 'sta']])
# orderLog = orderLog[orderLog['mrstaat'].isin([11000, 13000, 1000, 3000])]
# display(orderLog[orderLog['updateType'] == 0].groupby(['mrstaat', 'mrstauc'])['date'].size())

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
106636,20201214,zt_96_09,965801,1601222,-1,7,-1.0,0,0,-1.0,8.240000e+02
295137,20201214,zs_94_03,9461,2002044,-1,7,34444.0,0,0,-1.0,-1.000000e+00
330239,20201214,zs_64_01,6480,2002293,-1,1,47893.0,0,0,-1.0,1.038220e+05
335644,20201214,zs_52_06,5269,2002329,-1,1,37688.0,0,0,-1.0,6.962440e+05
342409,20201214,zs_52_09,5291,2002390,-1,1,51351.0,0,0,-1.0,1.331409e+06
346335,20201214,zs_64_01,6480,2002411,-1,1,53411.0,0,0,-1.0,1.516870e+05
354862,20201214,zs_54_01,5474,2002472,-1,1,47572.0,0,0,-1.0,8.689118e+08
422367,20201214,zs_52_06,5269,2002845,-1,1,39762.0,0,0,-1.0,9.018290e+05
431413,20201214,zs_52_06,5269,2002890,-1,1,37463.0,0,0,-1.0,6.683230e+05
539711,20201214,zs_52_06,5269,2300410,-1,1,35229.0,0,0,-1.0,3.052970e+05


There are orders with all things same except sdd
         ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
1726162   3686264.0   0.00044                    0           100   
1726443  18252604.0  0.001406                    0           100   
1758144   3688220.0  0.000419                    0           100   
1758167  14404537.0  0.000657                    0           100   

         absOrderSizeCumFilled  accCode  ars  caamd  cancellationPending  \
1726162                      0   966301  0.0      0                  0.0   
1726443                      0   966301  0.0      0                  0.0   
1758144                      0   966301  0.0      0                  0.0   
1758167                      0   966301  0.0      0                  0.0   

               cfe         clock    clockAtArrival  cumSharesBought  \
1726162  506777.82  1.608083e+15  1608083268955951            100.0   
1726443  147834.78  1.608101e+15  1608101490951270              0.0   
1758144  568630.

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1]:
import pandas as pd
pd.read_csv(r'\\192.168.10.34\trading\Kevin_Zhenyu_Kirk\makingOrder1214_1217.csv')

,Unnamed: 0,date,innerSeq,ID,accCode,updateType,order,time,mta_60s,mtmPrice_60s,mta_180s,mtmPrice_180s,mta_300s,mtmPrice_300s
0,3596,20201214.0,3596.0,2300379,5456.0,0.0,54501.0,2020-12-14 09:31:22.900041,0.001099,38.255,0.001089,38.195,0.001715,38.535
1,2392,20201214.0,2392.0,2002832,5456.0,0.0,54146.0,2020-12-14 09:31:23.050028,0.002766,15.775,0.002414,15.855,0.002637,15.935
2,2393,20201214.0,2393.0,2002832,5456.0,4.0,54146.0,2020-12-14 09:31:23.365403,0.002766,15.775,0.002414,15.855,0.002637,15.935
3,2394,20201214.0,2394.0,2002832,5456.0,0.0,54147.0,2020-12-14 09:31:23.840020,0.002766,15.775,0.002414,15.855,0.002637,15.935
4,3597,20201214.0,3597.0,2300379,5456.0,4.0,54501.0,2020-12-14 09:31:23.880543,0.001099,38.255,0.001089,38.195,0.001715,38.535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21480,16401,20201217.0,16401.0,1603338,9441.0,4.0,729829.0,2020-12-17 14:56:23.271768,-0.003145,104.400,-0.004591,104.400,-0.002955,104.380
21481,16402,20201217.0,16402.0,1603338,9441.0,0.0,729830.0,2020-12-17 14:56:31.000116,-0.003145,104.400,-0.004591,104.400,-0.002955,104.380
21482,16516,20201217.0,16516.0,1603355,9441.0,3.0,729882.0,2020-12-17 14:56:32.655871,0.001154,32.360,0.000969,32.490,0.000958,32.510
21483,15842,20201217.0,15842.0,1600640,9441.0,0.0,729179.0,2020-12-17 14:56:33.000062,0.000838,13.430,0.001245,13.430,0.001338,13.435


In [14]:
orderLog[(orderLog['order'] == 54501)][['date', 'secid', 'accCode', 'updateType', 'absOrderSize', 'absFilledThisUpdate', 'orderPrice', 'tradePrice', 'mta', 'mrm', 'mra100', 'mrb100']]

,date,secid,accCode,updateType,absOrderSize,absFilledThisUpdate,orderPrice,tradePrice,mta,mrm,mra100,mrb100
213862,20201214,2300379,5456,0,500,0,38.73,-1.00,-0.000257,-0.000916,3874.0,3866.0
213863,20201214,2300379,5456,2,500,0,38.73,-1.00,-1.000000,-0.000916,3874.0,3866.0
213864,20201214,2300379,5456,4,500,100,38.73,38.73,-1.000000,-0.000862,3872.0,3866.0
213865,20201214,2300379,5456,1,500,0,38.73,-1.00,-1.000000,-0.000661,3866.0,3862.0
213866,20201214,2300379,5456,3,500,0,38.73,-1.00,-1.000000,-0.000820,3867.0,3866.0


In [4]:
data = pd.read_pickle(r'\\192.168.10.34\trading\Kevin_Zhenyu_Kirk\1214_1217_making.pkl')
data[data['order'] == 54501]

,date,innerSeq,secid,accCode,updateType,order,time
3596,20201214,3596,2300379,5456,0,54501,2020-12-14 09:31:22.900041
3597,20201214,3597,2300379,5456,4,54501,2020-12-14 09:31:23.880543
3598,20201214,3598,2300379,5456,3,54501,2020-12-14 09:31:29.087752


In [5]:
IF_group = pd.read_table(r'E:\stockGroup_IF.txt', header=None)
IC_group = pd.read_table(r'E:\stockGroup_IC.txt', header=None)
CSI1000_group = pd.read_table(r'E:\stockGroup_CSI1000.txt', header=None)
IF_group1 = IF_group.iloc[0, :].values[0][1:-2].split(', ')
IF_group2 = IF_group.iloc[1, :].values[0][1:-2].split(', ')
IC_group1 = IC_group.iloc[0, :].values[0][1:-2].split(', ')
IC_group2 = IC_group.iloc[1, :].values[0][1:-2].split(', ')
CSI1000_group1 = CSI1000_group.iloc[0, :].values[0][1:-2].split(', ')
CSI1000_group2 = CSI1000_group.iloc[1, :].values[0][1:-2].split(', ')
assert(len(IF_group1) == len(IF_group2))
assert(len(IC_group1) == len(IC_group2))
assert(len(CSI1000_group1) == len(CSI1000_group2))

test1 = orderLog[orderLog['secid'].isin(IF_group1 + IF_group2)]
test1['index_name'] = 'IF'
test1['group_tag'] = np.where(test1['secid'].isin(IF_group1), 1, 2)
display(test1[(test1['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test1[(test1['updateType'] == 0) & (test1['accCode'] == 9461)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test2 = orderLog[orderLog['secid'].isin(IC_group1 + IC_group2)]
test2['index_name'] = 'IC'
test2['group_tag'] = np.where(test2['secid'].isin(IC_group1), 1, 2)
display(test2[(test2['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test2[(test2['updateType'] == 0) & (test2['accCode'] == 9441)].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())
test3 = orderLog[orderLog['secid'].isin(CSI1000_group1 + CSI1000_group2)]
test3['index_name'] = 'CSI1000'
test3['group_tag'] = np.where(test3['secid'].isin(CSI1000_group1), 1, 2)
display(test3[(test3['updateType'] == 0)].groupby(['colo', 'accCode', 'group_tag'])['ars'].unique())
display(test3[(test3['updateType'] == 0) & (test3['accCode'].isin([5456, 9758]))].groupby(['colo', 'accCode', 'group_tag'])['secid'].nunique())

test = pd.concat([test2[(test2['accCode'] == 9441) & (test2['date'] >= 20201210)], test3[test3['accCode'].isin([5456, 9758])], test1[test1['accCode'] == 9461]])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


colo      accCode  group_tag
zs_52_06  5384     1                 [1.0]
                   2                 [1.0]
zs_52_08  5281     1                 [1.0]
                   2                 [1.0]
zs_52_09  5264     1                 [1.0]
                   2                 [1.0]
zs_88_04  8967     1                 [1.0]
                   2                 [1.0]
          892402   1                 [1.0]
                   2                 [1.0]
zs_94_03  9461     1                 [2.0]
                   2            [1.0, 7.0]
zs_94_05  9435     1                 [1.0]
                   2                 [1.0]
zs_96_02  9655     1                 [1.0]
                   2                 [1.0]
zs_96_06  9765     1                 [1.0]
                   2                 [1.0]
zt_88_06  8970     1                 [1.0]
                   2                 [1.0]
          8971     1                 [1.0]
                   2                 [1.0]
          897102   2     

colo      accCode  group_tag
zs_94_03  9461     1            35
                   2            34
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5332     1                 [1.0]
                   2                 [1.0]
          5384     1                 [1.0]
                   2                 [1.0]
zs_52_08  5229     1                 [1.0]
                   2                 [1.0]
          5276     1                 [1.0]
                   2                 [1.0]
          5292     1                 [1.0]
                   2                 [1.0]
          5377     1                 [1.0]
                   2                 [1.0]
          5386     1                 [1.0]
                   2                 [1.0]
zs_52_09  5289     1                 [1.0]
                   2                 [1.0]
          5291     1                 [1.0]
                   2                 [1.0]
zs_72_01  7293     1                 [1.0]
                   2                 [1.0]
zs_88_04  8856     1                 [1.0]
                   2                 [1.0]
          8924     1     

colo      accCode  group_tag
zt_94_06  9441     1            38
                   2            31
Name: secid, dtype: int64

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


colo      accCode  group_tag
zs_52_06  5232     1                      [1.0]
                   2                      [1.0]
          5275     1                      [1.0]
                   2                      [1.0]
          5287     1                      [1.0]
                   2                      [1.0]
          5384     1                      [1.0]
                   2                      [1.0]
zs_54_01  5456     1                 [3.0, 2.0]
                   2                 [7.0, 1.0]
zs_66_01  6623     1                      [1.0]
                   2                      [1.0]
          6627     1                      [1.0]
                   2                      [1.0]
          6683     1                      [1.0]
                   2                      [1.0]
zs_88_04  8865     1                      [1.0]
                   2                      [1.0]
zs_92_02  9243     1                      [1.0]
                   2                      [1.0]
zs_94_05  9

colo      accCode  group_tag
zs_54_01  5456     1            106
                   2             93
zs_96_06  9758     1            104
                   2            101
Name: secid, dtype: int64

In [36]:
test1 = orderLog[(orderLog['secid'].isin(IC_group1 + IC_group2)) & (orderLog['accCode'] == 9441)]
test1 = test1[(test1['updateType'] == 4) & (test1['ars'] == 2) & (test1['orderDirection'].isin([1, 2])) & (test1['date'] >= 20201210)]
test1['mid'] = (test1['mra100'] + test1['mrb100'])/200
test1['tp'] = (test1['mrm'] + 1) * test1['mid']
test1['mrm_return'] = (test1['tp']/test1['tradePrice'] - 1) * 10000
display(test1['mrm_return'].describe())
display((test1['mrm_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())
test1 = orderLog[(orderLog['secid'].isin(IC_group1 + IC_group2)) & (orderLog['accCode'] == 9441)]
test1 = test1[(test1['updateType'] == 4) & (test1['ars'] == 3) & (test1['orderDirection'].isin([1, 2])) & (test1['date'] >= 20201210)]
test1['mid'] = (test1['mra100'] + test1['mrb100'])/200
test1['tp'] = (test1['mrm'] + 1) * test1['mid']
test1['mrm_return'] = (test1['tp']/test1['tradePrice'] - 1) * 10000
display(test1['mrm_return'].describe())
display((test1['mrm_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())
test1 = orderLog[(orderLog['secid'].isin(IC_group1 + IC_group2)) & (orderLog['accCode'] == 9441)]
test1 = test1[(test1['updateType'] == 4) & (test1['ars'] == 1) & (test1['orderDirection'].isin([1, 2])) & (test1['date'] >= 20201210)]
test1['mid'] = (test1['mra100'] + test1['mrb100'])/200
test1['tp'] = (test1['mrm'] + 1) * test1['mid']
test1['mrm_return'] = (test1['tp']/test1['tradePrice'] - 1) * 10000
display(test1['mrm_return'].describe())
display((test1['mrm_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())

count    680.000000
mean      26.412712
std        8.421019
min      -23.307539
25%       20.937511
50%       25.218986
75%       30.325667
max       72.906675
Name: mrm_return, dtype: float64

26.096277745369488

count    528.000000
mean      22.771679
std        7.605225
min       -2.043955
25%       18.175190
50%       21.904189
75%       25.451060
max       68.596842
Name: mrm_return, dtype: float64

22.546146377002838

count    464.000000
mean      19.106009
std        8.763083
min      -14.864183
25%       13.989245
50%       19.256696
75%       24.441824
max       51.061093
Name: mrm_return, dtype: float64

15.511427910619517

In [64]:
orderLog[(orderLog['secid'] == 1603868) & (orderLog['internalId'] == 1836) & (orderLog['secid'] == 1603868)][['updateType', 'secid', 'internalId', 'orderId', 'orderDirection', 'ars', 'vai', 'absOrderSize', 'absFilledThisUpdate', 'tradePrice']]

,updateType,secid,internalId,orderId,orderDirection,ars,vai,absOrderSize,absFilledThisUpdate,tradePrice


In [63]:
rawOrderLog[(rawOrderLog['secid'] == 1603868) & (rawOrderLog['internalId'] == 1836) & (rawOrderLog['secid'] == 1603868)][['updateType', 'secid', 'internalId', 'orderId', 'orderDirection', 'ars', 'vai', 'absOrderSize', 'absFilledThisUpdate', 'tradePrice']]

,updateType,secid,internalId,orderId,orderDirection,ars,vai,absOrderSize,absFilledThisUpdate,tradePrice
4954454,2,1603868,1836.0,2393290.0,1,-1.0,-1,600,0,-1.00
4954455,4,1603868,1836.0,2393290.0,1,-1.0,-1,600,200,49.94
4954456,4,1603868,1836.0,2393290.0,1,-1.0,-1,600,200,49.94
4954457,4,1603868,1836.0,2393290.0,1,-1.0,-1,600,100,49.94
4954738,0,1603868,1836.0,-1.0,1,1.0,744013,600,0,-1.00


In [40]:
test1 = orderLog[(orderLog['secid'].isin(IC_group1 + IC_group2)) & (orderLog['accCode'] == 9441)]
test1 = test1[(test1['updateType'] == 4) & (test1['ars'] == 2) & (test1['orderDirection'].isin([1, 2])) & (test1['date'] >= 20201210)]
test1['F90s_return'] = (test1['adjMid_F90s']/test1['tradePrice'] - 1) * 10000
test1['F300s_return'] = (test1['adjMid_F300s']/test1['tradePrice'] - 1) * 10000
display(test1['F90s_return'].describe())
display((test1['F90s_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())
display(test1['F300s_return'].describe())
display((test1['F300s_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())

test1 = orderLog[(orderLog['secid'].isin(IC_group1 + IC_group2)) & (orderLog['accCode'] == 9441)]
test1 = test1[(test1['updateType'] == 4) & (test1['ars'] == 3) & (test1['orderDirection'].isin([1, 2])) & (test1['date'] >= 20201210)]
test1['F90s_return'] = (test1['adjMid_F90s']/test1['tradePrice'] - 1) * 10000
test1['F300s_return'] = (test1['adjMid_F300s']/test1['tradePrice'] - 1) * 10000
display(test1['F90s_return'].describe())
display((test1['F90s_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())
display(test1['F300s_return'].describe())
display((test1['F300s_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())

test1 = orderLog[(orderLog['secid'].isin(IC_group1 + IC_group2)) & (orderLog['accCode'] == 9441)]
test1 = test1[(test1['updateType'] == 4) & (test1['ars'] == 1) & (test1['orderDirection'].isin([1, 2])) & (test1['date'] >= 20201210)]
test1['F90s_return'] = (test1['adjMid_F90s']/test1['tradePrice'] - 1) * 10000
test1['F300s_return'] = (test1['adjMid_F300s']/test1['tradePrice'] - 1) * 10000
display(test1['F90s_return'].describe())
display((test1['F90s_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())
display(test1['F300s_return'].describe())
display((test1['F300s_return'] * test1['tradeNtl']).sum() / test1['tradeNtl'].sum())

count    680.000000
mean      11.977887
std       35.092444
min      -59.141013
25%       -1.990136
50%       11.406356
75%       23.804801
max      591.843752
Name: F90s_return, dtype: float64

13.676859760166545

count    680.000000
mean      15.703936
std       49.186460
min     -113.169978
25%       -9.784259
50%       12.526096
75%       34.622548
max      326.033696
Name: F300s_return, dtype: float64

18.603523927491718

count    528.000000
mean      13.326949
std       41.083380
min      -43.457046
25%        0.791161
50%        9.328440
75%       20.390329
max      587.127440
Name: F90s_return, dtype: float64

...

14.694672421617534

count    464.000000
mean      17.123946
std       40.752609
min     -102.321538
25%        0.077892
50%       11.815588
75%       22.802907
max      332.613391
Name: F90s_return, dtype: float64

26.32285089894063

count    464.000000
mean      19.240861
std       46.798103
min     -125.768364
25%       -1.511330
50%       11.543381
75%       30.611571
max      333.509794
Name: F300s_return, dtype: float64

28.084065071629986

In [6]:
test = test[(test['ars'].isin([1, 2, 3]))]
test['order_type'] = np.where(test['ars'] == 1, 'take', np.where(
test['ars'] == 2, 'improved-NBBO make', 'on-NBBO make'))
# test['order_type'] = np.where(test['ars'] == 1, 'take', 'make')
test['Side'] = np.where(test['orderDirection'] == 1, 'Buy', 'Sell')
test['duration'] = np.where((test['date'] >= 20201210) & (test['accCode'] == 9441), '20201210-20201216', '20201207-20201216')
f1 = test[(test['updateType'] == 0)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['orderNtl'].sum().reset_index()
f2 = test[(test['updateType'] == 4)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].sum().reset_index()
f = pd.merge(f1, f2, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
f['fill_rate'] = f['tradeNtl'] / f['orderNtl']
f1 = test[(test['updateType'] == 0)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['order'].nunique().reset_index()
f2 = test[(test['updateType'] == 3)].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['order'].nunique().reset_index()
ff = pd.merge(f1, f2, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff['cancel_rate'] = ff['order_y'] / ff['order_x']
f = pd.merge(f, ff[['duration', 'Side', 'order_type', 'cancel_rate', 'accCode', 'index_name']], on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
test['cancel_duration_med'] = test['clockAtArrival'] - test['startClock']
test['cancel_duration_90p'] = test['clockAtArrival'] - test['startClock']
ff = test[test['updateType'] == 3].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['cancel_duration_med'].median().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff = test[test['updateType'] == 3].groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['cancel_duration_90p'].quantile(0.9).reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
test1 = test[test['updateType'] == 4]
test1['mid'] = (test1['mra100'] + test1['mrb100'])/200
test1['tp'] = (test1['mrm'] + 1) * test1['mid']
test1['mrm_return'] = np.where(test1['Side'] == 'Sell', test1['tradePrice']/test1['tp'] - 1, test1['tp']/test1['tradePrice'] - 1)
test1['sumNtl'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
test1['sum_return'] = test1['mrm_return'] * test1['tradeNtl']
test1['sum_return'] = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
test1['mrm_return'] = test1['sum_return'] / test1['sumNtl']
ff = test1.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['mrm_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
from IPython.display import display, HTML
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'order_type']).first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

In [7]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)

import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML
# 上次遗留问题：调整图间间距
# 调整标签位置，不要和图中重合
startDate = '20201207'
endDate = '20201216'

readPath = "L:\\orderLog\\ret\\***"
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
df = []
add = read_memb_daily(db1, 'index_memb', int(startDate), int(endDate), index_id=[1000300, 1000905, 1000852, 1000985])
add = add.rename(columns={'skey':'secid'})

dateLs = np.sort(dateLs)
dataPathLs = np.sort(dataPathLs)
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_pickle(thisPath)
    data = data.iloc[:, 1:]
    data = data.sort_values(by=['order', 'innerSeq'])
    if 'mrstaat' in data.columns:
        data["mrstaat"] = data.groupby(['order'])['mrstaat'].transform('first')
        data["mrstauc"] = data.groupby(['order'])['mrstauc'].transform('first')
        data["mrsb90"] = data.groupby(['order'])['mrsb90'].transform('first')
        data["mrss90"] = data.groupby(['order'])['mrss90'].transform('first')
        data["aaa"] = data.groupby(['order'])['aaa'].transform('first')
    data["ars"] = data.groupby(['order'])['ars'].transform('first')
    data["sdd"] = data.groupby(['order'])['sdd'].transform('first')
    
    data = data[data['updateType'] == 4]
    data = data[data['secid'].isin(add['secid'].unique())]
    print('1. Loading Data ' + thisDate )
    if data[data['absFilledThisUpdate'] == 0].shape[0] != 0:
        print('There are stocks with zero trade size')
        print(data[data['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
        data = data[data['absFilledThisUpdate'] != 0]
    if data[data['beta_60'].isnull()].shape[0] != 0:
        print('There are stocks with null beta')
        print(data[data['beta_60'].isnull()])
        data = data[~data['beta_60'].isnull()]
    data['max_trade'] = data.groupby('order')['absOrderSizeCumFilled'].transform('max')
    data['last'] = 0
    data.loc[data['max_trade'] == data['absOrderSizeCumFilled'], 'last'] = 1

    data["buyRet"] = np.where(data["orderDirection"].isin([1, 2]), data["adjMid_F90s"] / data["tradePrice"] - 1, np.nan)
    data["buyRet1"] = np.where(data["orderDirection"].isin([1, 2]), data["adjMid_F300s"] / data["tradePrice"] - 1, np.nan)
    data["sellRet"] = np.where(data["orderDirection"].isin([-1, -2]), data["tradePrice"] / data["adjMid_F90s"] - 1, np.nan)
    data["sellRet1"] = np.where(data["orderDirection"].isin([-1, -2]), data["tradePrice"] / data["adjMid_F300s"] - 1, np.nan)
    data["indexRet"] = data["indexClose_F90s"]/data["indexClose"] - 1
    data["indexRet1"] = data["indexClose_F300s"]/data["indexClose"] - 1
    data["buyAlpha"] = np.where(data["orderDirection"].isin([1, 2]), data["buyRet"] - data["indexRet"] * data["beta_60"], np.nan)
    data["buyAlpha1"] = np.where(data["orderDirection"].isin([1, 2]), data["buyRet1"] - data["indexRet1"] * data["beta_60"], np.nan)
    data["sellAlpha"] = np.where(data["orderDirection"].isin([-1, -2]), data["sellRet"] + data["indexRet"] * data["beta_60"], np.nan)
    data["sellAlpha1"] = np.where(data["orderDirection"].isin([-1, -2]), data["sellRet1"] + data["indexRet1"] * data["beta_60"], np.nan)
    data["buyNum"] = np.where((data["orderDirection"].isin([1, 2])) & (data['last'] == 1), 1, 0)
    data["sellNum"] = np.where((data["orderDirection"].isin([-1, -2])) & (data['last'] == 1), 1, 0)
    data['server_account'] = data['colo'].apply(lambda x: x.replace('_', '')) + '_' + data['accCode'].astype(str)
    df += [data]
df = pd.concat(df, sort=False)

print('2. Orders with nan ars in updateType 0')
if df[df['ars'].isnull()].shape[0] != 0:
    print(df[df['ars'].isnull()])
    df = df[~df['ars'].isnull()]
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([121, 131])), 'mrstauc'] = 1000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([221, 231])), 'mrstauc'] = 2000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([321, 331])), 'mrstauc'] = 3000
df.loc[(df['date'] <= 20201013) & (~df['ars'].isin([121, 221, 321, 131, 231, 331])), 'mrstauc'] = 0
df.loc[(df['date'] <= 20201013) & (df['mrstauc'].isin([1000, 2000, 3000])), 'mrstaat'] = 3000
df.loc[(df['date'] <= 20201013) & (df['mrstauc'] == 0), 'mrstaat'] = 1000

print('3. Current strategy status')
if 2 in df['mrstauc'].unique():
    print(df[df['mrstauc'] == 2].groupby('mrstaat')['mrstauc'].unique())
    df.loc[df['mrstauc'] == 2, 'mrstauc'] = 0
print(df.groupby('mrstaat')['mrstauc'].unique())

print('4. Strange strategy')
if df[(df["mrstaat"].isnull()) | (df['mrstaat'] == 0) | (df['mrstaat'] == 2000)].shape[0] != 0:
    print('1) null mrstaat value')
    print(df[(df["mrstaat"].isnull())][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print(df[(df["mrstaat"].isnull())].groupby(['date', 'colo', 'accCode'])['secid'].size())
    print('2) 0 mrstaat value')
    print(df[(df["mrstaat"] == 0)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print('3) 2000 mrstaat value')
    print(df[(df["mrstaat"] == 2000)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
df = df[(~df["mrstaat"].isnull()) & (df['mrstaat'] != 0) & (df['mrstaat'] != 2000)]

if 'mrsb90' in df.columns:
    df['m1'] = df['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    df['m2'] = df['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if df[df['mrsb90'] == '-'].shape[0] != 0:
        print('There are ticks with mrsb90 value -')
        print(df[df['mrsb90'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['mrsb90'] != '-']
    if df[df['aaa'] == '-'].shape[0] != 0:
        print('There are ticks with aaa value -')
        print(df[df['aaa'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['aaa'] != '-']   
    df['mrsb90'] = df['mrsb90'].astype(float)
    df['aaa'] = df['aaa'].astype(float)
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm1']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm1']   

df = pd.merge(df, add[['date', 'secid', 'index_name']], on=['date', 'secid'], how='left')

1. Loading Data 20201207
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
5358    0.0                112.0         112.0                  112.0   
17459   0.0                112.0         112.0                  112.0   
33748   0.0                112.0         112.0                  112.0   
46400   0.0                112.0         112.0                  112.0   
59842   0.0                112.0         112.0                  112.0   
64334   0.0                112.0         112.0                  112.0   
88413   0.0                112.0         112.0                  112.0   
100565  0.0                112.0         112.0                  112.0   
113534  0.0                 93.0         112.0                   93.0   
113535  0.0                 19.0         112.0                  112.0   
132169  0.0                112.0         112.0                  112.0   
138994  0.0                112.0         112.0                  112

1. Loading Data 20201208
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
5028    0.0                112.0         112.0                  112.0   
20646   0.0                112.0         112.0                  112.0   
30662   0.0                112.0         112.0                  112.0   
43413   0.0                112.0         112.0                  112.0   
52517   0.0                112.0         112.0                  112.0   
66945   0.0                112.0         112.0                  112.0   
70655   0.0                112.0         112.0                  112.0   
73777   0.0                112.0         112.0                  112.0   
91897   0.0                112.0         112.0                  112.0   
101725  0.0                112.0         112.0                  112.0   
112777  0.0                112.0         112.0                  112.0   
129526  0.0                112.0         112.0                  112

1. Loading Data 20201209
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
370435  0.0                129.0         129.0                  129.0   
371658  0.0                129.0         129.0                  129.0   
505370  0.0                129.0         129.0                  129.0   
513597  0.0                129.0         129.0                  129.0   
523956  0.0                129.0         129.0                  129.0   
530651  0.0                129.0         129.0                  129.0   
536064  0.0                129.0         129.0                  129.0   
545609  0.0                129.0         129.0                  129.0   
551723  0.0                129.0         129.0                  129.0   
560948  0.0                129.0         129.0                  129.0   
570808  0.0                129.0         129.0                  129.0   
581967  0.0                129.0         129.0                  129

1. Loading Data 20201210
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
323859  0.0                288.0         288.0                  288.0   
446822  0.0               1313.0        1313.0                 1313.0   
452052  0.0               1313.0        1313.0                 1313.0   
464752  0.0               1313.0        1313.0                 1313.0   
470010  0.0               1313.0        1313.0                 1313.0   
481122  0.0               1313.0        1313.0                 1313.0   
490860  0.0                690.0        1313.0                  690.0   
490861  0.0                200.0        1313.0                  890.0   
490862  0.0                423.0        1313.0                 1313.0   
498888  0.0               1296.0        1296.0                 1296.0   
511037  0.0                935.0         935.0                  935.0   
583246  0.0                200.0         200.0                  200

1. Loading Data 20201211
There are stocks with zero trade size
colo      accCode
zs_88_04  8967       1
Name: secid, dtype: int64
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
282860  0.0                520.0         724.0                  520.0   
282861  0.0                204.0         724.0                  724.0   
316396  0.0                724.0         724.0                  724.0   
328846  0.0                724.0         724.0                  724.0   
337695  0.0                128.0         724.0                  128.0   
337696  0.0                100.0         724.0                  228.0   
337697  0.0                200.0         724.0                  428.0   
337698  0.0                200.0         724.0                  628.0   
337699  0.0                 96.0         724.0                  724.0   
350711  0.0                176.0         724.0                  176.0   
350712  0.0                400.0   

1. Loading Data 20201214
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
325450  0.0                228.0         228.0                  228.0   
325453  0.0                813.0         813.0                  813.0   
329217  0.0                228.0         228.0                  228.0   
329220  0.0               1254.0        1399.0                 1254.0   
329221  0.0                145.0        1399.0                 1399.0   
529153  0.0                  4.0         228.0                    4.0   
529154  0.0                224.0         228.0                  228.0   
529157  0.0               2504.0        2504.0                 2504.0   
529160  0.0                 75.0        2780.0                   75.0   
529161  0.0                  1.0        2780.0                   76.0   
529162  0.0                200.0        2780.0                  276.0   
544421  0.0                228.0         228.0                  228

1. Loading Data 20201215
There are stocks with zero trade size
colo      accCode
zs_88_04  8967       2
Name: secid, dtype: int64
1. Loading Data 20201216
There are stocks with zero trade size
colo      accCode
zs_88_04  8967       2
Name: secid, dtype: int64
There are stocks with null beta
        aaa  absFilledThisUpdate  absOrderSize  absOrderSizeCumFilled  \
250340    0                112.0         112.0                  112.0   
258036    0                112.0         112.0                  112.0   
281974    0                112.0         112.0                  112.0   
294342    0                112.0         112.0                  112.0   
303449    0                112.0         112.0                  112.0   
315151    0                112.0         112.0                  112.0   
327804    0                112.0         112.0                  112.0   
334587    0                112.0         112.0                  112.0   
371960    0                129.0        1940.0     

2. Orders with nan ars in updateType 0
3. Current strategy status
mrstaat
0.0                                                    [0.0]
1000.0                                                 [0.0]
2000.0                                              [1000.0]
3000.0                              [1000.0, 2000.0, 3000.0]
11000.0                          [10000.0, 20000.0, 30000.0]
13000.0    [11000.0, 12000.0, 21000.0, 31000.0, 23000.0, ...
Name: mrstauc, dtype: object
4. Strange strategy
1) null mrstaat value
Empty DataFrame
Columns: [date, secid, vai, sdd, absFilledThisUpdate, colo, accCode, mrstaat, mrstauc, ars]
Index: []
Series([], Name: secid, dtype: int64)
2) 0 mrstaat value
            date      secid        vai          sdd  absFilledThisUpdate  \
715686  20201216  1600021.0  1221426.0  111922000.0                200.0   
715689  20201216  1600021.0  1223026.0  111922000.0                200.0   
715692  20201216  1600021.0  1227326.0  111923000.0                500.0   
715695  20

In [5]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)
add = read_memb_daily(db1, 'index_memb', int(startDate), int(endDate), index_id=[1000300, 1000905, 1000852, 1000985])
add = add.rename(columns={'skey':'secid'})

import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML
# 上次遗留问题：调整图间间距
# 调整标签位置，不要和图中重合

if 'mrstaat' in df.columns:
    df["mrstaat"] = df.groupby(['order'])['mrstaat'].transform('first')
    df["mrstauc"] = df.groupby(['order'])['mrstauc'].transform('first')
    df["mrsb90"] = df.groupby(['order'])['mrsb90'].transform('first')
    df["mrss90"] = df.groupby(['order'])['mrss90'].transform('first')
    df["aaa"] = df.groupby(['order'])['aaa'].transform('first')
df["ars"] = df.groupby(['order'])['ars'].transform('first')
df["sdd"] = df.groupby(['order'])['sdd'].transform('first')

df = df[df['updateType'] == 4]
df = df[df['secid'].isin(add['secid'].unique())]
if df[df['absFilledThisUpdate'] == 0].shape[0] != 0:
    print('There are stocks with zero trade size')
    print(df[df['absFilledThisUpdate'] == 0].groupby(['colo', 'accCode'])['secid'].size())
    df = df[df['absFilledThisUpdate'] != 0]
if df[df['beta_60'].isnull()].shape[0] != 0:
    print('There are stocks with null beta')
    print(df[df['beta_60'].isnull()])
    df = df[~df['beta_60'].isnull()]
df['max_trade'] = df.groupby('order')['absOrderSizeCumFilled'].transform('max')
df['last'] = 0
df.loc[df['max_trade'] == df['absOrderSizeCumFilled'], 'last'] = 1

df["buyRet"] = np.where(df["orderDirection"].isin([1, 2]), df["adjMid_F90s"] / df["tradePrice"] - 1, np.nan)
df["buyRet1"] = np.where(df["orderDirection"].isin([1, 2]), df["adjMid_F300s"] / df["tradePrice"] - 1, np.nan)
df["sellRet"] = np.where(df["orderDirection"].isin([-1, -2]), df["tradePrice"] / df["adjMid_F90s"] - 1, np.nan)
df["sellRet1"] = np.where(df["orderDirection"].isin([-1, -2]), df["tradePrice"] / df["adjMid_F300s"] - 1, np.nan)
df["indexRet"] = df["indexClose_F90s"]/df["indexClose"] - 1
df["indexRet1"] = df["indexClose_F300s"]/df["indexClose"] - 1
df["buyAlpha"] = np.where(df["orderDirection"].isin([1, 2]), df["buyRet"] - df["indexRet"] * df["beta_60"], np.nan)
df["buyAlpha1"] = np.where(df["orderDirection"].isin([1, 2]), df["buyRet1"] - df["indexRet1"] * df["beta_60"], np.nan)
df["sellAlpha"] = np.where(df["orderDirection"].isin([-1, -2]), df["sellRet"] + df["indexRet"] * df["beta_60"], np.nan)
df["sellAlpha1"] = np.where(df["orderDirection"].isin([-1, -2]), df["sellRet1"] + df["indexRet1"] * df["beta_60"], np.nan)
df["buyNum"] = np.where((df["orderDirection"].isin([1, 2])) & (df['last'] == 1), 1, 0)
df["sellNum"] = np.where((df["orderDirection"].isin([-1, -2])) & (df['last'] == 1), 1, 0)
df['server_account'] = df['colo'].apply(lambda x: x.replace('_', '')) + '_' + df['accCode'].astype(str)

print('2. Orders with nan ars in updateType 0')
if df[df['ars'].isnull()].shape[0] != 0:
    print(df[df['ars'].isnull()])
    df = df[~df['ars'].isnull()]
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([121, 131])), 'mrstauc'] = 1000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([221, 231])), 'mrstauc'] = 2000
df.loc[(df['date'] <= 20201013) & (df['ars'].isin([321, 331])), 'mrstauc'] = 3000
df.loc[(df['date'] <= 20201013) & (~df['ars'].isin([121, 221, 321, 131, 231, 331])), 'mrstauc'] = 0
df.loc[(df['date'] <= 20201013) & (df['mrstauc'].isin([1000, 2000, 3000])), 'mrstaat'] = 3000
df.loc[(df['date'] <= 20201013) & (df['mrstauc'] == 0), 'mrstaat'] = 1000

print('3. Current strategy status')
if 2 in df['mrstauc'].unique():
    print(df[df['mrstauc'] == 2].groupby('mrstaat')['mrstauc'].unique())
    df.loc[df['mrstauc'] == 2, 'mrstauc'] = 0
print(df.groupby('mrstaat')['mrstauc'].unique())

print('4. Strange strategy')
if df[(df["mrstaat"].isnull()) | (df['mrstaat'] == 0) | (df['mrstaat'] == 2000)].shape[0] != 0:
    print('1) null mrstaat value')
    print(df[(df["mrstaat"].isnull())][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print(df[(df["mrstaat"].isnull())].groupby(['date', 'colo', 'accCode'])['secid'].size())
    print('2) 0 mrstaat value')
    print(df[(df["mrstaat"] == 0)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
    print('3) 2000 mrstaat value')
    print(df[(df["mrstaat"] == 2000)][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'ars']])
df = df[(~df["mrstaat"].isnull()) & (df['mrstaat'] != 0) & (df['mrstaat'] != 2000)]

if 'mrsb90' in df.columns:
    df['m1'] = df['mrstaat'].apply(lambda x: x - (x // 10000) * 10000)
    df['m2'] = df['mrstauc'].apply(lambda x: x - (x // 10000) * 10000)
    if df[df['mrsb90'] == '-'].shape[0] != 0:
        print('There are ticks with mrsb90 value -')
        print(df[df['mrsb90'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['mrsb90'] != '-']
    if df[df['aaa'] == '-'].shape[0] != 0:
        print('There are ticks with aaa value -')
        print(df[df['aaa'] == '-'][['date', 'secid', 'vai', 'sdd', 'absFilledThisUpdate', 'colo', 'accCode', 'mrstaat', 'mrstauc', 'aaa', 'ars']])
        df = df[df['aaa'] != '-']   
    df['mrsb90'] = df['mrsb90'].astype(float)
    df['aaa'] = df['aaa'].astype(float)
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] >= 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrsb90']) < 1e-12), 'm1']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstauc'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm2']

    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'mrstaat'] = \
    df.loc[(df['orderDirection'] < 1) &\
             (df['mrstaat'].isin([11000, 13000])) & (abs(df['aaa'] - df['mrss90']) < 1e-12), 'm1']   

df = pd.merge(df, add[['date', 'secid', 'index_name']], on=['date', 'secid'], how='left')

There are stocks with zero trade size
colo      accCode
zs_88_04  8967       5
          896702     1
Name: secid, dtype: int64
There are stocks with null beta
         ApplSeqNum       aaa  absFilledThisUpdate  absOrderSize  \
41642           0.0  0.001614                  500           800   
41643           0.0  0.001614                  100           800   
41644           0.0  0.001614                  200           800   
715809          0.0  0.001687                  300          1000   
715810          0.0  0.001687                  400          1000   
715811          0.0  0.001687                  300          1000   
715814          0.0  0.001687                  100          1000   
715815          0.0  0.001687                  100          1000   
729161          0.0  0.000838                  200           200   
885431          0.0  0.002689                  900           900   
889658          0.0  0.002021                  600           600   
906194          0.0  0.0

2. Orders with nan ars in updateType 0
3. Current strategy status
mrstaat
0.0                                                    [0.0]
1000.0                                                 [0.0]
2000.0                                              [1000.0]
3000.0                              [1000.0, 2000.0, 3000.0]
11000.0                          [10000.0, 20000.0, 30000.0]
13000.0    [11000.0, 21000.0, 22000.0, 12000.0, 31000.0, ...
Name: mrstauc, dtype: object
4. Strange strategy
1) null mrstaat value
Empty DataFrame
Columns: [date, secid, vai, sdd, absFilledThisUpdate, colo, accCode, mrstaat, mrstauc, ars]
Index: []
Series([], Name: secid, dtype: int64)
2) 0 mrstaat value
             date    secid        vai          sdd  absFilledThisUpdate  \
5449085  20201216  1600021  1221426.0  111922000.0                  200   
5449088  20201216  1600021  1223026.0  111922000.0                  200   
5449091  20201216  1600021  1227326.0  111923000.0                  500   
5449094  20201

In [47]:
test = pd.read_csv(r'\\192.168.10.28\equityTradeLogs\speedCompare_20201216.csv')

In [51]:
test[(test['updateType'] == 0) & (test['mrstaat'] == 0) & (test['accCode'] == 966301)][['updateType', 'colo', 'accCode', 'secid', 'vai', 'ars', 'ApplSeqNum', 'sdd', 'orderPrice', 'absOrderSize', 'absFilledThisUpdate', 'mrstaat', 'mrstauc']]

,updateType,colo,accCode,secid,vai,ars,ApplSeqNum,sdd,orderPrice,absOrderSize,absFilledThisUpdate,mrstaat,mrstauc
678983,0.0,zt_96_01,966301,1600315.0,2248491.0,0.0,0.0,94528000.0,40.71,100.0,0.0,0.0,0.0
678986,0.0,zt_96_01,966301,2300207.0,5049405.0,0.0,3337408.0,94529690.0,29.22,600.0,0.0,0.0,0.0
678988,0.0,zt_96_01,966301,1600126.0,1507500.0,0.0,0.0,94530000.0,5.23,100.0,0.0,0.0,0.0
678992,0.0,zt_96_01,966301,2300595.0,1017668.0,0.0,3562064.0,94531700.0,78.55,100.0,0.0,0.0,0.0
678994,0.0,zt_96_01,966301,2002423.0,702000.0,0.0,3505749.0,94531990.0,9.39,200.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
681724,0.0,zt_96_01,966301,2002572.0,5424157.0,0.0,18252604.0,145131110.0,26.25,100.0,0.0,0.0,0.0
681728,0.0,zt_96_01,966301,2000686.0,14929904.0,0.0,19239901.0,145341170.0,9.50,100.0,0.0,0.0,0.0
681731,0.0,zt_96_01,966301,2000686.0,14936304.0,0.0,19242076.0,145342580.0,9.50,100.0,0.0,0.0,0.0
681734,0.0,zt_96_01,966301,2000681.0,4290883.0,0.0,19274374.0,145557710.0,14.67,100.0,0.0,0.0,0.0


In [53]:
test[(test['updateType'] == 0) & (test['mrstaat'] == 0) & (test['accCode'] == 966301)]['sdd'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]).astype('int64')

count          872
mean     103939346
std       13140115
min       94528000
10%       94722214
20%       94949450
30%       95266960
40%       95926938
50%      100518165
60%      101510744
70%      103114957
80%      105817248
90%      130904700
max      145631000
Name: sdd, dtype: int64

In [11]:
df3 = df[((df['secid'].isin(CSI1000_group1 + CSI1000_group2)) & (df['accCode'].isin([5456, 9758]))) | \
        ((df['secid'].isin(IC_group1 + IC_group2)) & (df['accCode'] == 9441) & (df['date'] >= 20201210)) | \
        ((df['secid'].isin(IF_group1 + IF_group2)) & (df['accCode'] == 9461))]
rdf = df3[df3['ars'].isin([1, 2, 3])]
rdf['index_name'] = np.where(rdf['accCode'] == 9461, 'IF', np.where(
rdf['accCode'] == 9441, 'IC', 'CSI1000'))
rdf['order_type'] = np.where(rdf['ars'] == 1, 'take', np.where(
rdf['ars'] == 2, 'improved-NBBO make', 'on-NBBO make'))
# rdf['order_type'] = np.where(rdf['ars'] == 1, 'take', 'make')
rdf['duration'] = np.where((rdf['date'] >= 20201210) & (rdf['accCode'] == 9441), '20201210-20201216', '20201207-20201216')
rdf['tradeNtl'] = rdf['tradePrice']*rdf['absFilledThisUpdate']
rdf['Side'] = np.where(rdf['orderDirection'] == 1, 'Buy', 'Sell')
rdf['F90s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet'], rdf['sellRet'])
rdf['F300s_return'] = np.where(rdf['Side'] == 'Buy', rdf['buyRet1'], rdf['sellRet1'])

rdf['sumNtl'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['tradeNtl'].transform(sum)
rdf['sum_return'] = rdf['F90s_return'] * rdf['tradeNtl']
rdf['sum_return'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return'].transform(sum)
rdf['F90s_return'] = rdf['sum_return'] / rdf['sumNtl']
rdf['sum_return1'] = rdf['F300s_return'] * rdf['tradeNtl']
rdf['sum_return1'] = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['sum_return1'].transform(sum)
rdf['F300s_return'] = rdf['sum_return1'] / rdf['sumNtl']
ff = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['F90s_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
ff = rdf.groupby(['duration', 'Side', 'order_type', 'accCode', 'index_name'])['F300s_return'].first().reset_index()
f = pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])
HTML(f.groupby(['duration', 'Side',  'index_name', 'accCode', 'order_type']).first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [69]:
pd.merge(f, ff, on=['duration', 'Side', 'order_type', 'accCode', 'index_name'])

,duration,Side,order_type,accCode,index_name,orderNtl,tradeNtl,fill_rate,cancel_rate,cancel_duration_med,cancel_duration_90p,mrm_return_x,mrm_return_y
0,20201209-20201211,Buy,improved-NBBO make,5456,CSI1000,2916004.00,1200848.00,0.411813,0.636000,5891985.0,42482219.2,0.002207,0.002207
1,20201209-20201211,Buy,on-NBBO make,5456,CSI1000,5449825.00,1548895.75,0.284210,0.704100,14531053.0,69321932.6,0.001512,0.001512
2,20201209-20201211,Buy,take,5456,CSI1000,2234841.00,1927857.12,0.862637,0.129187,1516345.5,4174970.8,0.000800,0.000800
3,20201209-20201211,Sell,improved-NBBO make,5456,CSI1000,4322145.00,1223146.00,0.282995,0.697872,9950081.5,61731496.7,0.001728,0.001728
4,20201209-20201211,Sell,on-NBBO make,5456,CSI1000,5786562.00,1325577.32,0.229079,0.765661,13405197.0,60729307.4,0.001315,0.001315
5,20201209-20201211,Sell,take,5456,CSI1000,1985176.66,1681699.66,0.847128,0.093333,1897426.5,5100322.1,0.000281,0.000281
6,20201210-20201211,Buy,improved-NBBO make,9441,IC,13359283.00,5102182.00,0.381920,0.627986,15117422.0,60275135.4,0.002651,0.002651
7,20201210-20201211,Buy,on-NBBO make,9441,IC,17091001.00,4161230.00,0.243475,0.734793,18525295.5,80329931.4,0.002285,0.002285
8,20201210-20201211,Buy,take,9441,IC,11221325.00,4483685.00,0.399568,0.424419,3164015.0,6176781.2,0.001174,0.001174
9,20201210-20201211,Sell,improved-NBBO make,9441,IC,12254626.00,4472987.00,0.365004,0.644144,15810419.0,81037501.2,0.001420,0.001420


In [68]:
f.shape[0]

12

In [64]:
ff.head(1)

,duration,Side,order_type,accCode,index_name,mrm_return
0,20201209-20201211,Buy,improved-NBBO make,5456,CSI1000,0.002207


In [49]:
rdf[rdf['duration'] == '20201210-20201211']['accCode'].unique()

array([9441], dtype=int64)

In [12]:
for i in ['fill_rate', 'cancel_rate']:
    f[i] = f[i].apply(lambda x: '%.1f%%'%(x*100))
for i in ['orderNtl', 'tradeNtl']:
    f[i] = (f[i]/1000).astype(int)
for i in ['cancel_duration_med', 'cancel_duration_90p']:
    f[i] = (f[i]/1000000).apply(lambda x: '%.0f'%(x))
for i in ['mrm_return', 'F90s_return', 'F300s_return']:
     f[i] = f[i].apply(lambda x: '%.2f'%(x*10000))
f

,duration,Side,order_type,accCode,index_name,orderNtl,tradeNtl,fill_rate,cancel_rate,cancel_duration_med,cancel_duration_90p,mrm_return,F90s_return,F300s_return
0,20201207-20201216,Buy,improved-NBBO make,5456,CSI1000,9568,3328,34.8%,68.3%,7,44,18.12,17.60,19.54
1,20201207-20201216,Buy,improved-NBBO make,9461,IF,76915,35107,45.6%,57.7%,6,35,11.74,12.90,9.09
2,20201207-20201216,Buy,improved-NBBO make,9758,CSI1000,77845,27863,35.8%,65.8%,9,52,15.89,14.81,13.75
3,20201207-20201216,Buy,on-NBBO make,5456,CSI1000,16273,4392,27.0%,73.1%,13,67,14.25,8.87,11.84
4,20201207-20201216,Buy,take,5456,CSI1000,6596,5644,85.6%,13.7%,2,5,6.09,22.13,15.06
5,20201207-20201216,Buy,take,9461,IF,34416,33351,96.9%,4.9%,2,3,7.88,15.54,9.71
6,20201207-20201216,Buy,take,9758,CSI1000,44150,32975,74.7%,25.3%,2,5,6.83,21.05,15.81
7,20201207-20201216,Sell,improved-NBBO make,5456,CSI1000,10935,3226,29.5%,69.4%,9,52,16.12,9.84,-0.88
8,20201207-20201216,Sell,improved-NBBO make,9461,IF,77259,34306,44.4%,58.9%,7,37,9.22,12.30,17.90
9,20201207-20201216,Sell,improved-NBBO make,9758,CSI1000,67687,27719,41.0%,63.6%,12,69,12.55,9.16,4.42


In [13]:
f['stockGroup'] = np.where(f['order_type'].isin(['improved-NBBO make', 'on-NBBO make']), 'groupA', 'groupB')
f = f.rename(columns={'order_type':'orderType', 'fill_rate':'fillRate', 'cancel_rate':'cancelRate', 'mrm_return':'Mta'})
HTML(f.groupby(['duration', 'index_name', 'accCode', 'Side',  'stockGroup', 'orderType'])['orderNtl', 'tradeNtl', 'cancelRate', 'cancel_duration_med', 'cancel_duration_90p', 'fillRate', 'Mta', 'F90s_return', 'F300s_return'].first().to_html())

In [14]:
test = pd.read_pickle(r'L:\orderLog\data\20201216.pkl')

In [17]:
test[(~test['beta_60'].isnull()) & (test['mrstaat'].isnull())]

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s
40511,NaN,-1,3000,4500,4000,897002,NaN,1608101662655972,0.0,1.090000e+02,1.608102e+15,1608101663341375,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,0.0,813.0,24000.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:54:23.340857,0,-1.0,NaN,NaN,-1,8.130000e+02,1.0,9.14,NaN,NaN,1600388,114591230.0,0,24534.0,1022.0,209.0,NaN,9.14,905,4,0,logs_20201216_zt_88_03_day_897002,zt_88_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.785063,9.130823,9.112110,9.110000,6232.2065,6232.4502,6233.2449,6237.1943
73314,NaN,0.001219,0,500,0,965801,NaN,1608101799865248,0.0,4.133381e+05,1.608102e+15,1608101799865397,NaN,NaN,NaN,NaN,20201216,0.0,4.133371e+05,0.001000,0.0,1.0,1038.0,7900.0,6500.0,0.0,0.0,NaN,NaN,0.0,0.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:39.864480,0,0.0,NaN,1.510140e+07,1,1.038000e+03,0.0,24.45,NaN,145639000.0,1600685,122773461.0,0,2230.0,1205.0,168.0,NaN,-1.00,852,0,0,logs_20201216_zt_96_09_day_965801,zt_96_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.934630,24.945385,24.833973,24.950000,6597.3116,6594.3972,6596.5788,6589.3329
90173,NaN,-1,200,900,200,8971,NaN,1608101807257590,0.0,1.600000e+01,1.608102e+15,1608101807618482,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,0.0,1908.0,17453.0,2000.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:47.617418,0,-1.0,NaN,NaN,1,1.908000e+03,1.0,300.00,NaN,NaN,1600809,116277554.0,0,440370.0,2513.0,650.0,NaN,300.00,300,4,0,logs_20201216_zt_88_06_day_8971longshort,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.259699,299.980000,299.980000,299.993867,4953.9132,4954.9413,4954.9413,4953.8670
177351,NaN,-1,0,100,0,9551,NaN,1608101733807318,0.0,1.050835e+07,1.608102e+15,1608101733891190,NaN,NaN,NaN,NaN,20201216,0.0,1.050835e+07,NaN,0.0,0.0,1636.0,9201.0,9200.0,0.0,0.0,NaN,NaN,0.0,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:55:33.890839,0,-1.0,NaN,6.900000e+01,1,2.003710e+06,1.0,34.08,NaN,NaN,1603456,123207537.0,0,434444.0,2344.0,708.0,NaN,-1.00,852,2,0,logs_20201216_zt_94_06_day_9551,zt_94_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.056159,34.110000,34.160000,34.150099,6542.0107,6542.2068,6544.4474,6543.3624
207062,NaN,0.001,0,100,0,968501,NaN,1608101800279630,0.0,NaN,1.608102e+15,1608101800279815,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,1.0,828.0,28500.0,28500.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:40.278479,0,NaN,NaN,NaN,1,-1.000000e+00,0.0,26.12,NaN,NaN,1603733,122780119.0,0,2152.0,1096.0,268.0,NaN,-1.00,852,0,0,logs_20201216_zt_96_09_day_968501,zt_96_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.857278,26.078824,26.292213,26.612963,6597.3116,6594.8625,6596.1687,6588.6166
286447,NaN,-1,1800,16600,1880,6623,NaN,1608101762322571,0.0,1.185469e+07,1.608102e+15,1608101762322784,NaN,NaN,NaN,NaN,20201216,0.0,1.185469e+07,NaN,0.0,0.0,3711.0,24380.0,24380.0,0.0,0.0,NaN,NaN,0.0,0.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:02.321640,0,-1.0,NaN,8.802799e+07,1,1.812811e+10,1.0,7.04,NaN,NaN,2000851,210700601.0,0,83850.0,3711.0,586.0,NaN,7.04,852,4,0,logs_20201216_zs_66_01_day_6623,zs_66_01,NaN,NaN

In [70]:
orderLog[(orderLog['ars'] == 7) & (orderLog['accCode'] == 5456)]

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status,orderNtl,m1,m2,exchange,tradeNtl,sta
249125,12317818.0,-0.000231,0,100,0,5456,7.0,1607403874148068,0.0,76395.0418,2020-12-08 13:04:34.148106,1607403874148106,800.0,1000.0,100.0,0.0,20201208,0.0,76394.0418,-0.000100,0.0,1.0,1235.0,2800.0,800.0,0.0,0.0,0.0,0.0,0.0,-1.0,1851.0,1850.0,-0.000289,-0.000615,1.0,0.000545,-1.0,-0.000304,-1.0,-0.000231,1000.0,0.0,0.0,6309770.0,13:04:34.147885,100,0.0,-0.000904,5125673.88,-1,-1.0,0.0,18.50,NaN,130426740.0,2002123,86851832.0,0,11058.0,1235.0,384.0,NaN,-1.00,852,0,6309770.0,logs_20201208_zs_54_01_day_5456,zs_54_01,36452,54,zs_54,36455,35603,1607403874148106,0,-1,1,1.0,0,1850.0,1000.0,0.0,SZE,0.0,1. staone
249126,0.0,-0.000231,0,100,0,5456,7.0,1607403874151989,0.0,76395.0418,2020-12-08 13:04:34.152048,1607403874152048,800.0,1000.0,100.0,0.0,20201208,0.0,76394.0418,-1.000000,0.0,0.0,1235.0,2800.0,800.0,0.0,0.0,0.0,0.0,0.0,-1.0,1851.0,1850.0,-0.000289,-0.000615,1.0,0.000545,-1.0,-0.000304,-1.0,-0.000231,1000.0,0.0,0.0,6309870.0,13:04:34.151885,0,-1.0,-1.000000,5125673.88,-1,868909303.0,1.0,18.50,00C0QKZS8M02G1JR,-1.0,2002123,86851843.0,0,11085.0,1235.0,384.0,NaN,-1.00,852,2,6309770.0,logs_20201208_zs_54_01_day_5456,zs_54_01,36452,54,zs_54,36455,35603,1607403874148106,3942,-1,1,1.0,0,1850.0,1000.0,0.0,SZE,0.0,1. staone
249127,0.0,-0.000231,100,100,100,5456,7.0,1607403874151989,0.0,78245.0418,2020-12-08 13:04:34.152090,1607403874152090,800.0,1000.0,100.0,100.0,20201208,1.0,78244.0418,-1.000000,0.0,0.0,1235.0,2700.0,800.0,0.0,0.0,0.0,0.0,0.0,-1.0,1851.0,1850.0,-0.000289,-0.000615,1.0,0.000545,-1.0,-0.000304,-1.0,-0.000231,1000.0,0.0,0.0,6309870.0,13:04:34.151885,0,-1.0,-1.000000,5125673.88,-1,868909303.0,0.0,18.50,NaN,-1.0,2002123,86851843.0,0,11085.0,1235.0,384.0,1.01e+14,18.50,852,4,6309770.0,logs_20201208_zs_54_01_day_5456,zs_54_01,36452,54,zs_54,36455,35603,1607403874148106,3984,-1,1,1.0,0,1850.0,1000.0,0.0,SZE,1850.0,1. staone
249128,12320276.0,-0.000129,0,100,0,5456,7.0,1607403877611062,0.0,78238.1918,2020-12-08 13:04:37.611081,1607403877611081,800.0,1000.0,200.0,100.0,20201208,0.0,78237.1918,-0.000100,0.0,1.0,1236.0,2700.0,800.0,0.0,0.0,0.0,0.0,0.0,-1.0,1851.0,1850.0,-0.000289,-0.000615,1.0,0.000545,-1.0,-0.000406,-1.0,-0.000129,1000.0,0.0,0.0,6313670.0,13:04:37.610885,100,0.0,-0.000904,5123497.88,-1,-1.0,0.0,18.50,NaN,130430200.0,2002123,86871803.0,0,11058.0,1236.0,384.0,NaN,-1.00,852,0,6313670.0,logs_20201208_zs_54_01_day_5456,zs_54_01,36453,54,zs_54,36456,35604,1607403877611081,0,-1,1,1.0,0,1850.0,1000.0,0.0,SZE,0.0,1. staone
249129,0.0,-0.000129,0,100,0,5456,7.0,1607403877614910,0.0,78238.1918,2020-12-08 13:04:37.614922,1607403877614922,800.0,1000.0,200.0,100.0,20201208,0.0,78237.1918,-1.000000,0.0,0.0,1236.0,2700.0,800.0,0.0,0.0,0.0,0.0,0.0,-1.0,1851.0,1850.0,-0.000289,-0.000615,1.0,0.000545,-1.0,-0.000406,-1.0,-0.000129,1000.0,0.0,0.0,6313770.0,13:04:37.613885,0,-1.0,-1.000000,5123497.88,-1,868909305.0,1.0,18.50,00C0QKZS8M02G20J,-1.0,2002123,86871817.0,0,11085.0,1236.0,384.0,NaN,-1.00,852,2,6313670.0,logs_20201208_zs_54_01_day_5456,zs_54_01,36453,54,zs_54,36456,35604,1607403877611081,3841,-1,1,1.0,0,1850.0,1000.0,0.0,SZE,0.0,1. staone
249130,0.0,-0.000129,100,100,100,5456,7.0,1607403877614910,0.0,80088.1918,2020-12-08 13:04:37.614964,1607403877614964,800.0

In [85]:
orderLog[(orderLog['updateType'] == 0) & (orderLog['ars'] == 7)].groupby(['date'])['accCode'].unique()

date
20201209          [9441]
20201210    [9441, 5456]
20201211          [9441]
Name: accCode, dtype: object

In [86]:
test = pd.concat([orderLog[(orderLog['accCode'].isin([9441])) & (orderLog['ars'].isin([1, 7]))], 
                  orderLog[(orderLog['accCode'].isin([5456])) & (orderLog['ars'].isin([1, 7])) & (orderLog['date'] == 20201210)]])
re1 = test[test['updateType'] == 0].groupby(['date', 'accCode'])['order'].size().reset_index()
re1 = re1.rename(columns={'order':'totalNum'})
re2 = test[(test['updateType'] == 0) & (test['ars'] == 7)].groupby(['date', 'accCode'])['order'].size().reset_index()
re2 = re2.rename(columns={'order':'IFRNum'})
re = pd.merge(re2, re1, on=['date', 'accCode'])
re['perc(%)'] = re['IFRNum'] / re['totalNum']
re['perc(%)'] = re['perc(%)'].apply(lambda x: '%.f'%(x * 100))
HTML(re.groupby(['date', 'accCode']).first().to_html())

In [87]:
test['tag'] = np.where(test['ars'] == 1, '2. normal taking orders', np.where(
test['ars'] == 7, '1. IFR taking orders', 'else'))
test['Side'] = np.where(test['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
display(test[test['tag'] == 'else'])
re1 = test[test['updateType'] == 0].groupby(['date', 'Side', 'accCode', 'tag'])['order'].size().reset_index()
re1 = re1.rename(columns={'order':'numOfTotalOrders'})
re2 = test[test['updateType'] == 1].groupby(['date', 'Side', 'accCode', 'tag'])['order'].size().reset_index()
re2 = re2.rename(columns={'order':'numOfCancelOrders'})
re = pd.merge(re1, re2, on=['date', 'Side', 'accCode', 'tag'], how='left')
re = re.fillna(0)
re['fillRate(%)'] = 1 - re['numOfCancelOrders'] / re['numOfTotalOrders']
re['fillRate(%)'] = re['fillRate(%)'].apply(lambda x: '%.1f'%(x * 100))
ree = re
HTML(re.groupby(['date', 'Side', 'accCode', 'tag']).first().to_html())

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status,orderNtl,m1,m2,exchange,tradeNtl,sta,tag,Side


In [89]:
df1 = pd.concat([df[(df['accCode'].isin([9441])) & (df['ars'].isin([1, 7]))],
                 df[(df['accCode'].isin([5456])) & (df['ars'].isin([1, 7])) & (df['date'] == 20201210)]])
df1['tag'] = np.where(df1['ars'] == 1, '2. normal taking orders', np.where(
df1['ars'] == 7, '1. IFR taking orders', 'else'))
display(df1[df1['tag'] == 'else'])
df1['tradeNtl'] = df1['tradePrice'] * df1['absFilledThisUpdate']
df1['Side'] = np.where(df1['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
df1['stradeNtl'] = df1.groupby(['date', 'Side', 'accCode', 'tag'])['tradeNtl'].transform('sum')
df1['Ret'] = np.where(df1['Side'] == 'Buy', df1['buyRet'] * 10000, df1['sellRet'] * 10000)
df1['Ret1'] = np.where(df1['Side'] == 'Buy', df1['buyRet1'] * 10000, df1['sellRet1'] * 10000)
df1['90s_ret'] = df1['Ret'] * df1['tradeNtl']
df1['90s_ret']= df1.groupby(['date', 'Side', 'accCode', 'tag'])['90s_ret'].transform('sum')
df1['90s_ret']= df1['90s_ret'] / df1['stradeNtl']
df1['300s_ret'] = df1['Ret1'] * df1['tradeNtl']
df1['300s_ret']= df1.groupby(['date', 'Side', 'accCode', 'tag'])['300s_ret'].transform('sum')
df1['300s_ret']= df1['300s_ret'] / df1['stradeNtl']
re1 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['order'].nunique().reset_index()
re1 = re1.rename(columns={'order':'numOfOrders'})
re2 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['tradeNtl'].sum().reset_index()
re2 = re2.rename(columns={'tradeNtl':'notional'})
re2['notional_sum'] = re2.groupby(['date', 'Side', 'accCode'])['notional'].transform('sum')
re2['notionalPerc'] = (re2['notional'] / re2['notional_sum']) * 100
re3 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['90s_ret'].first().reset_index()
re4 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['300s_ret'].first().reset_index()
re = pd.merge(re1, re2, on=['date', 'Side', 'accCode', 'tag'])
re = pd.merge(re, re3, on=['date', 'Side', 'accCode', 'tag'])
re = pd.merge(re, re4, on=['date', 'Side', 'accCode', 'tag'])
re['notional'] = re['notional'].astype(int)
for cols in ['90s_ret', '300s_ret']:
    re[cols] = re[cols].apply(lambda x: '%.2f'%(x))
for cols in ['notionalPerc']:
    re[cols] = re[cols].apply(lambda x: '%.1f'%(x))
re = pd.merge(re, ree[['date', 'Side', 'accCode', 'tag', 'fillRate(%)']], on=['date', 'Side', 'accCode', 'tag'])
fr = re
HTML(re.groupby(['date', 'Side', 'accCode', 'tag'])[['numOfOrders', 'notional', 'notionalPerc', 'fillRate(%)', '90s_ret', '300s_ret']].first().to_html())

,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,max_trade,last,buyRet,buyRet1,sellRet,sellRet1,indexRet,indexRet1,buyAlpha,buyAlpha1,sellAlpha,sellAlpha1,buyNum,sellNum,server_account,m1,m2,index_name,tag


In [90]:
test['tag'] = '3. all orders'
test['Side'] = np.where(test['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
display(test[test['tag'] == 'else'])
re1 = test[test['updateType'] == 0].groupby(['date', 'Side', 'accCode', 'tag'])['order'].size().reset_index()
re1 = re1.rename(columns={'order':'numOfTotalOrders'})
re2 = test[test['updateType'] == 1].groupby(['date', 'Side', 'accCode', 'tag'])['order'].size().reset_index()
re2 = re2.rename(columns={'order':'numOfCancelOrders'})
re = pd.merge(re1, re2, on=['date', 'Side', 'accCode', 'tag'], how='left')
re = re.fillna(0)
re['fillRate(%)'] = 1 - re['numOfCancelOrders'] / re['numOfTotalOrders']
re['fillRate(%)'] = re['fillRate(%)'].apply(lambda x: '%.1f'%(x * 100))
ree1 = re
HTML(re.groupby(['date', 'Side', 'accCode', 'tag']).first().to_html())

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status,orderNtl,m1,m2,exchange,tradeNtl,sta,tag,Side


In [91]:
df1 = pd.concat([df[(df['accCode'].isin([9441])) & (df['ars'].isin([1, 7]))],
                 df[(df['accCode'].isin([5456])) & (df['ars'].isin([1, 7])) & (df['date'] == 20201210)]])
df1['tag'] = '3. all orders'
display(df1[df1['tag'] == 'else'])
df1['tradeNtl'] = df1['tradePrice'] * df1['absFilledThisUpdate']
df1['Side'] = np.where(df1['orderDirection'].isin([1, 2]), 'Buy', 'Sell')
df1['stradeNtl'] = df1.groupby(['date', 'Side', 'accCode', 'tag'])['tradeNtl'].transform('sum')
df1['Ret'] = np.where(df1['Side'] == 'Buy', df1['buyRet'] * 10000, df1['sellRet'] * 10000)
df1['Ret1'] = np.where(df1['Side'] == 'Buy', df1['buyRet1'] * 10000, df1['sellRet1'] * 10000)
df1['90s_ret'] = df1['Ret'] * df1['tradeNtl']
df1['90s_ret']= df1.groupby(['date', 'Side', 'accCode', 'tag'])['90s_ret'].transform('sum')
df1['90s_ret']= df1['90s_ret'] / df1['stradeNtl']
df1['300s_ret'] = df1['Ret1'] * df1['tradeNtl']
df1['300s_ret']= df1.groupby(['date', 'Side', 'accCode', 'tag'])['300s_ret'].transform('sum')
df1['300s_ret']= df1['300s_ret'] / df1['stradeNtl']
re1 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['order'].nunique().reset_index()
re1 = re1.rename(columns={'order':'numOfOrders'})
re2 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['tradeNtl'].sum().reset_index()
re2 = re2.rename(columns={'tradeNtl':'notional'})
re2['notional_sum'] = re2.groupby(['date', 'Side', 'accCode'])['notional'].transform('sum')
re2['notionalPerc'] = (re2['notional'] / re2['notional_sum']) * 100
re3 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['90s_ret'].first().reset_index()
re4 = df1.groupby(['date', 'Side', 'accCode', 'tag'])['300s_ret'].first().reset_index()
re = pd.merge(re1, re2, on=['date', 'Side', 'accCode', 'tag'])
re = pd.merge(re, re3, on=['date', 'Side', 'accCode', 'tag'])
re = pd.merge(re, re4, on=['date', 'Side', 'accCode', 'tag'])
re['notional'] = re['notional'].astype(int)
for cols in ['90s_ret', '300s_ret']:
    re[cols] = re[cols].apply(lambda x: '%.2f'%(x))
for cols in ['notionalPerc']:
    re[cols] = re[cols].apply(lambda x: '%.1f'%(x))
re = pd.merge(re, ree1[['date', 'Side', 'accCode', 'tag', 'fillRate(%)']], on=['date', 'Side', 'accCode', 'tag'])
re1 = re
HTML(re.groupby(['date', 'Side', 'accCode', 'tag'])[['numOfOrders', 'notional', 'notionalPerc', 'fillRate(%)', '90s_ret', '300s_ret']].first().to_html())

,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,test,order,firstUpdateType,insertNum,innerSeq,firstClock,indexCat,beta_60,mdStartClock,mdClockAtArrival,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s,max_trade,last,buyRet,buyRet1,sellRet,sellRet1,indexRet,indexRet1,buyAlpha,buyAlpha1,sellAlpha,sellAlpha1,buyNum,sellNum,server_account,m1,m2,index_name,tag


In [92]:
fr = pd.concat([fr, re1])
fr = fr.sort_values(by=['date', 'Side', 'accCode', 'tag'])
fr.index = pd.MultiIndex.from_arrays([list(fr['date'].values), list(fr['Side'].values), list(fr['accCode'].values), list(fr['tag'].values)],
                                            names=['date', 'Side', 'accCode', 'tag'])
fr.index = pd.MultiIndex.from_tuples([(x[0], x[1], x[2], x[3][3:]) for x in fr.index],
                                            names=['date', 'Side', 'accCode', 'tag'])
HTML(fr[['numOfOrders', 'notional', 'notionalPerc', 'fillRate(%)', '90s_ret', '300s_ret']].to_html())

In [25]:
test[(~test['beta_60'].isnull()) & (test['ars'].isnull())]

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s
40511,NaN,-1,3000,4500,4000,897002,NaN,1608101662655972,0.0,1.090000e+02,1.608102e+15,1608101663341375,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,0.0,813.0,24000.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:54:23.340857,0,-1.0,NaN,NaN,-1,8.130000e+02,1.0,9.14,NaN,NaN,1600388,114591230.0,0,24534.0,1022.0,209.0,NaN,9.14,905,4,0,logs_20201216_zt_88_03_day_897002,zt_88_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.785063,9.130823,9.112110,9.110000,6232.2065,6232.4502,6233.2449,6237.1943
73314,NaN,0.001219,0,500,0,965801,NaN,1608101799865248,0.0,4.133381e+05,1.608102e+15,1608101799865397,NaN,NaN,NaN,NaN,20201216,0.0,4.133371e+05,0.001,0.0,1.0,1038.0,7900.0,6500.0,0.0,0.0,NaN,NaN,0.0,0.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:39.864480,0,0.0,NaN,1.510140e+07,1,1.038000e+03,0.0,24.45,NaN,145639000.0,1600685,122773461.0,0,2230.0,1205.0,168.0,NaN,-1.00,852,0,0,logs_20201216_zt_96_09_day_965801,zt_96_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.934630,24.945385,24.833973,24.950000,6597.3116,6594.3972,6596.5788,6589.3329
90173,NaN,-1,200,900,200,8971,NaN,1608101807257590,0.0,1.600000e+01,1.608102e+15,1608101807618482,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,0.0,1908.0,17453.0,2000.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:47.617418,0,-1.0,NaN,NaN,1,1.908000e+03,1.0,300.00,NaN,NaN,1600809,116277554.0,0,440370.0,2513.0,650.0,NaN,300.00,300,4,0,logs_20201216_zt_88_06_day_8971longshort,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.259699,299.980000,299.980000,299.993867,4953.9132,4954.9413,4954.9413,4953.8670
177351,NaN,-1,0,100,0,9551,NaN,1608101733807318,0.0,1.050835e+07,1.608102e+15,1608101733891190,NaN,NaN,NaN,NaN,20201216,0.0,1.050835e+07,NaN,0.0,0.0,1636.0,9201.0,9200.0,0.0,0.0,NaN,NaN,0.0,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:55:33.890839,0,-1.0,NaN,6.900000e+01,1,2.003710e+06,1.0,34.08,NaN,NaN,1603456,123207537.0,0,434444.0,2344.0,708.0,NaN,-1.00,852,2,0,logs_20201216_zt_94_06_day_9551,zt_94_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.056159,34.110000,34.160000,34.150099,6542.0107,6542.2068,6544.4474,6543.3624
207062,NaN,0.001,0,100,0,968501,NaN,1608101800279630,0.0,NaN,1.608102e+15,1608101800279815,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,1.0,828.0,28500.0,28500.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:40.278479,0,NaN,NaN,NaN,1,-1.000000e+00,0.0,26.12,NaN,NaN,1603733,122780119.0,0,2152.0,1096.0,268.0,NaN,-1.00,852,0,0,logs_20201216_zt_96_09_day_968501,zt_96_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.857278,26.078824,26.292213,26.612963,6597.3116,6594.8625,6596.1687,6588.6166
286447,NaN,-1,1800,16600,1880,6623,NaN,1608101762322571,0.0,1.185469e+07,1.608102e+15,1608101762322784,NaN,NaN,NaN,NaN,20201216,0.0,1.185469e+07,NaN,0.0,0.0,3711.0,24380.0,24380.0,0.0,0.0,NaN,NaN,0.0,0.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:02.321640,0,-1.0,NaN,8.802799e+07,1,1.812811e+10,1.0,7.04,NaN,NaN,2000851,210700601.0,0,83850.0,3711.0,586.0,NaN,7.04,852,4,0,logs_20201216_zs_66_01_day_6623,zs_66_01,NaN,NaN,Na

In [24]:
rawOrderLog[(~rawOrderLog['beta_60'].isnull()) & (rawOrderLog['ars'].isnull()) & (rawOrderLog['date'] == 20201216)]

,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,ars,caamd,cancellationPending,cfe,clock,clockAtArrival,cumSharesBought,cumSharesBuyInserted,cumSharesSellInserted,cumSharesSold,date,finalState,gfe,hee,insertedShortOrder,insertionPending,internalId,inv_L,inv_L0,inv_S,inv_S0,l4algoDebug,l4tr,locateShares,locateSharesTotal,mfe,mra100,mrb100,mrm,mrm25,mrmum,mrrlma,mrsb300,mrsb90,mrss300,mrss90,mrstaat,mrstauc,mrstaum,mrv,ms,mse,mt,mta,mv,orderDirection,orderId,orderOutstanding,orderPrice,orderSysId,sdd,secid,sequenceNo,session,threadId,totalActions,totalCanceled,tradeId,tradePrice,underlyingIndex,updateType,vai,zipFile,colo,caa_orderLog,start_time,Price,OrderQty,Side,statusLs,TradePriceLs,TradeQtyLs,beta_60,adjMid_F30s,adjMid_F90s,adjMid_F300s,indexClose,indexClose_F30s,indexClose_F90s,indexClose_F300s
4773680,NaN,-1,3000,4500,4000,897002,NaN,1608101662655972,0.0,1.090000e+02,1.608102e+15,1608101663341375,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,0.0,813.0,24000.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:54:23.340857,0,-1.0,NaN,NaN,-1,8.130000e+02,1.0,9.14,NaN,NaN,1600388,114591230.0,0,24534.0,1022.0,209.0,NaN,9.14,905,4,0,logs_20201216_zt_88_03_day_897002,zt_88_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.785063,9.130823,9.112110,9.110000,6232.2065,6232.4502,6233.2449,6237.1943
4806483,NaN,0.001219,0,500,0,965801,NaN,1608101799865248,0.0,4.133381e+05,1.608102e+15,1608101799865397,NaN,NaN,NaN,NaN,20201216,0.0,4.133371e+05,0.001,0.0,1.0,1038.0,7900.0,6500.0,0.0,0.0,NaN,NaN,0.0,0.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:39.864480,0,0.0,NaN,1.510140e+07,1,1.038000e+03,0.0,24.45,NaN,145639000.0,1600685,122773461.0,0,2230.0,1205.0,168.0,NaN,-1.00,852,0,0,logs_20201216_zt_96_09_day_965801,zt_96_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.934630,24.945385,24.833973,24.950000,6597.3116,6594.3972,6596.5788,6589.3329
4823342,NaN,-1,200,900,200,8971,NaN,1608101807257590,0.0,1.600000e+01,1.608102e+15,1608101807618482,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,0.0,1908.0,17453.0,2000.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:47.617418,0,-1.0,NaN,NaN,1,1.908000e+03,1.0,300.00,NaN,NaN,1600809,116277554.0,0,440370.0,2513.0,650.0,NaN,300.00,300,4,0,logs_20201216_zt_88_06_day_8971longshort,zt_88_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.259699,299.980000,299.980000,299.993867,4953.9132,4954.9413,4954.9413,4953.8670
4910520,NaN,-1,0,100,0,9551,NaN,1608101733807318,0.0,1.050835e+07,1.608102e+15,1608101733891190,NaN,NaN,NaN,NaN,20201216,0.0,1.050835e+07,NaN,0.0,0.0,1636.0,9201.0,9200.0,0.0,0.0,NaN,NaN,0.0,0.0,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:55:33.890839,0,-1.0,NaN,6.900000e+01,1,2.003710e+06,1.0,34.08,NaN,NaN,1603456,123207537.0,0,434444.0,2344.0,708.0,NaN,-1.00,852,2,0,logs_20201216_zt_94_06_day_9551,zt_94_06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.056159,34.110000,34.160000,34.150099,6542.0107,6542.2068,6544.4474,6543.3624
4940231,NaN,0.001,0,100,0,968501,NaN,1608101800279630,0.0,NaN,1.608102e+15,1608101800279815,NaN,NaN,NaN,NaN,20201216,0.0,NaN,NaN,0.0,1.0,828.0,28500.0,28500.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:40.278479,0,NaN,NaN,NaN,1,-1.000000e+00,0.0,26.12,NaN,NaN,1603733,122780119.0,0,2152.0,1096.0,268.0,NaN,-1.00,852,0,0,logs_20201216_zt_96_09_day_968501,zt_96_09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.857278,26.078824,26.292213,26.612963,6597.3116,6594.8625,6596.1687,6588.6166
5019616,NaN,-1,1800,16600,1880,6623,NaN,1608101762322571,0.0,1.185469e+07,1.608102e+15,1608101762322784,NaN,NaN,NaN,NaN,20201216,0.0,1.185469e+07,NaN,0.0,0.0,3711.0,24380.0,24380.0,0.0,0.0,NaN,NaN,0.0,0.0,-1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14:56:02.321640,0,-1.0,NaN,8.802799e+07,1,1.812811e+10,1.0,7.04,NaN,NaN,2000851,210700601.0,0,83850.0,3711.0,586.0,NaN,7.04,852,4,0,logs_20201216_zs_66_01_day_6623,zs_66_01,N

In [73]:
fr = fr.rename(columns={'notional':'notioanl(w)', 'notionalPerc':'notionalPerc(%)'})
# for i in ['notioanl(w)']:
#     fr[i] = (fr[i]/10000).apply(lambda x: '%.1f'%(x))
HTML(fr[['numOfOrders', 'notioanl(w)', 'notionalPerc(%)', 'fillRate(%)', '90s_ret', '300s_ret']].to_html())